<a href="https://colab.research.google.com/github/BatyaAbitbol/NogahProject/blob/Resnet18-TensorBoard/Nogah_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms

### **preproccessing the data**

In [3]:
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        return image, label

In [4]:
def data_preparation(data: pd.DataFrame):
    # Extract the pixel data and labels
    pixel_columns = [col for col in data.columns if col.startswith('pixel')]
    pixels = data[pixel_columns].values
    labels = data['label'].values

    # Reshape the pixel data to match the image dimensions (3, 32, 32)
    pixels = pixels.reshape(-1, 3, 32, 32)

    # Convert pixel data to float32 and normalize to the range [0, 1]
    pixels = pixels.astype(np.float32) / 255.0

    # Convert NumPy arrays to PyTorch tensors
    pixel_tensors = torch.tensor(pixels)
    label_tensors = torch.tensor(labels)

    # Standardize
    normalize_transform = transforms.Normalize(mean=[0.0, 0.0, 0.0], std=[1.0, 1.0, 1.0])
    pixel_tensors = normalize_transform(pixel_tensors)

    # Create a CustomDataset instance
    dataset = CustomDataset(pixel_tensors, label_tensors)

    # Create a DataLoader
    batch_size = 128
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    return loader

## Resnet18 model - Tensorboard

In [3]:
!pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [4]:
import torchvision.models as models
import torch.nn as nn
from tensorboardX import SummaryWriter

In [7]:
models.resnet18(pretrained=False, num_classes=13)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
def resnet18_model_with_tensorboard(num_epochs: int, train_loader, validation_loader):
    # Define the ResNet18 model
    model = models.resnet18(pretrained=False, num_classes=13)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9) ADAM

    # Set up TensorBoard writer
    writer = SummaryWriter('logs')

    #num_epochs = 10  # You can adjust this as needed

    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        total_loss = 0.0

        for i, (images, labels) in enumerate(train_loader):
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Average training loss for the epoch
        avg_loss = total_loss / len(train_loader)
        eval_loss = evaluate_model(evaluate_dataloader)

        # Write training loss to TensorBoard
        writer.add_scalar('Loss/train', avg_loss, epoch)
        witer.add_sclar('LOss/Val', eval_loss, epoch)

    # Close the TensorBoard writer
    writer.close()
    torch.save(model.state_dict(), '/content/drive/MyDrive/models/resnet18_model.pth')
    return model

In [8]:
df = pd.read_csv("/content/drive/MyDrive/Datasets/dataset.csv")
train_loader = prepare_the_data(df)
# test_loader = prepare_the_data(df) # Do the same for the test

In [9]:
model = resnet18_model_with_tensorboard(30, train_loader)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [10]:
def evaluate_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode

    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient calculation during evaluation
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [ ]:
accuracy = evaluate_model(model, test_loader)
print("Test Accuracy: {:.2f}%".format(accuracy * 100))

In [ ]:
# def load_model(model, model_path):
#     model.load_state_dict(torch.load(model_path))
#     model.eval()  # Set the model to evaluation mode after loading

# model = models.resnet18(pretrained=False)
# num_classes = 13
# model.fc = nn.Linear(512, num_classes)
# load_model(model, 'resnet18_model')

In [ ]:
tensorboard --logdir=logs

In [ ]:
class ResNet18_model():
  def __init__(self):
    pass

